In [224]:

import datetime
import pandas as pd
import yfinance as yf
import pandas_datareader.data as web
import numpy as np
import scipy as sp
from scipy.optimize import minimize


In [225]:
# FANG

jm = yf.download('JPM', start="2010-01-01", end="2023-05-01")
ap = yf.download('AAPL', start="2010-01-01", end="2023-05-01")
am = yf.download('AMZN', start="2010-01-01", end="2023-05-01")
wm = yf.download('WMT', start="2010-01-01", end="2023-05-01")
tg = yf.download('TGT', start="2010-01-01", end="2023-05-01")
ms = yf.download('MSFT', start="2010-01-01", end="2023-05-01")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [226]:
stocks = pd.concat([jm['Adj Close'], ap['Adj Close'], am['Adj Close'], wm['Adj Close'], tg['Adj Close'], ms['Adj Close']],axis=1)


In [227]:
stocks.columns = ['JP Morgan', 'Apple', 'Amazon', 'Walmart', 'Target', 'Microsoft']
stocks.tail()

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2023-04-24,140.729996,165.101639,106.209999,152.181900,162.837341,281.155365
2023-04-25,137.669998,163.543793,102.570000,150.996414,156.549911,274.819244
2023-04-26,135.229996,163.533798,104.980003,150.338913,154.404434,294.725708
2023-04-27,137.050003,168.177383,109.820000,150.807129,157.086273,304.165070
2023-04-28,138.240005,169.445618,105.449997,150.398682,156.688965,306.589783


In [228]:
stock_pd = pd.DataFrame(
    stocks
)

stock_pd.head()

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2010-01-04,30.279612,6.496294,6.6950,40.013077,34.240273,23.572369
2010-01-05,30.866135,6.507527,6.7345,39.614635,34.367241,23.579988
2010-01-06,31.035727,6.404015,6.6125,39.526085,35.023125,23.435276
2010-01-07,31.650488,6.392178,6.5000,39.548229,35.453320,23.191549
2010-01-08,31.572762,6.434672,6.6760,39.349014,35.312279,23.351494


In [229]:
stock_pd = stock_pd.fillna(stock_pd)
stock_pd.to_csv('stockprice.csv', index=True)

In [230]:
## Change in returns over time

returns = stocks/stocks.shift(1)
returns

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,1.019370,1.001729,1.005900,0.990042,1.003708,1.000323
2010-01-06,1.005494,0.984094,0.981884,0.997765,1.019085,0.993863
2010-01-07,1.019808,0.998152,0.982987,1.000560,1.012283,0.989600
2010-01-08,0.997544,1.006648,1.027077,0.994963,0.996022,1.006897
...,...,...,...,...,...,...
2023-04-24,1.001352,1.001879,0.992988,1.006788,1.010852,0.986037
2023-04-25,0.978256,0.990564,0.965728,0.992210,0.961388,0.977464
2023-04-26,0.982276,0.999939,1.023496,0.995646,0.986295,1.072435


In [231]:
## Find mean of returns

returns.mean()

JP Morgan    1.000614
Apple        1.001136
Amazon       1.001041
Walmart      1.000470
Target       1.000604
Microsoft    1.000901
dtype: float64

In [232]:
logReturns = np.log(returns)
logReturns

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.019185,0.001728,0.005883,-0.010008,0.003701,0.000323
2010-01-06,0.005479,-0.016034,-0.018282,-0.002238,0.018905,-0.006156
2010-01-07,0.019615,-0.001850,-0.017160,0.000560,0.012208,-0.010454
2010-01-08,-0.002459,0.006626,0.026717,-0.005050,-0.003986,0.006873
...,...,...,...,...,...,...
2023-04-24,0.001351,0.001877,-0.007037,0.006765,0.010794,-0.014061
2023-04-25,-0.021984,-0.009480,-0.034873,-0.007820,-0.039377,-0.022794
2023-04-26,-0.017883,-0.000061,0.023224,-0.004364,-0.013800,0.069932


In [233]:
pbar = logReturns.mean()
sigma = logReturns.cov()

In [234]:
rMin = 0.02
def riskFunction(w):
    return np.dot(w.T,np.dot(sigma,w))

w0 = [0.166, 0.166, 0.166, 0.166, 0.166, 0.167]
bounds = ((0,1), (0,1), (0,1), (0,1), (0,1), (0,1))

def checkMinimumRertun(w):
    RHS = rMin - np.sum(pbar * w)
    return RHS
def checkSumToOne(w):
    return np.sum(w) - 1 

constraints = ({'type': 'eq', 'fun': checkMinimumRertun},{'type': 'eq', 'fun': checkSumToOne})
w_opt = minimize(riskFunction, w0, method='SLSQP', bounds=bounds, constraints=constraints)

In [235]:
w_opt

 message: Positive directional derivative for linesearch
 success: False
  status: 8
     fun: 0.0003230395769952976
       x: [ 0.000e+00  9.971e-01  7.908e-12  0.000e+00  0.000e+00
            1.603e-11]
     nit: 10
     jac: [ 2.649e-04  6.480e-04  3.683e-04  1.336e-04  2.022e-04
            3.535e-04]
    nfev: 62
    njev: 6

In [236]:
w_scipy = w_opt.x
sum(w_scipy)

0.9970508523479518

In [237]:
risk_scipy = riskFunction(w_scipy)
risk_scipy

0.0003230395769952976

In [238]:
np.sum(pbar*w_scipy)


0.0009700722303484363

In [239]:
def markowitz(rMin,sigma,pbar):
    N = len(sigma)
    o = np.ones(N)
    sigmaInv = np.linalg.inv(sigma)
    a = np.dot(pbar.T, np.dot(sigmaInv,pbar))
    b = np.dot(pbar.T,np.dot(sigmaInv,o))
    c = np.dot(o.T,np.dot(sigmaInv,o))
    return (1/(a*c - b**2)) * np.dot(sigmaInv,((c*rMin - b)*pbar + (a-b*rMin)*o))

In [240]:
analysis = markowitz(rMin,sigma,pbar)
analysis

array([-13.43476274,  26.90798122,   4.24909223, -20.99099827,
        -4.60902852,   8.87771609])

In [241]:
stock_pd.head()

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2010-01-04,30.279612,6.496294,6.6950,40.013077,34.240273,23.572369
2010-01-05,30.866135,6.507527,6.7345,39.614635,34.367241,23.579988
2010-01-06,31.035727,6.404015,6.6125,39.526085,35.023125,23.435276
2010-01-07,31.650488,6.392178,6.5000,39.548229,35.453320,23.191549
2010-01-08,31.572762,6.434672,6.6760,39.349014,35.312279,23.351494


# Total Portfolio Return with optimal weight


In [242]:
#assume optimal stock portafolio
# JP Morgan	Apple	Amazon	Walmart	Target	Microsoft

w_optimal = [0.25381753, 0.01236147, 0.04539653, 0.54974009, 0.07737082, 0.06131356]

# Assume we invested $10,000 in 1/1/2010 and see the return on 4/30/2023
# also compare the return against only invest on single stock

stock_price_day1 = stock_pd[stock_pd.index == "2010-01-04"]
stock_price_day1

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2010-01-04,30.279612,6.496294,6.695,40.013077,34.240273,23.572369


In [243]:
shares_brought = w_optimal / stock_price_day1 * 10000


In [244]:
daily_price = stock_pd.mul(shares_brought.values[0])
daily_price


,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2010-01-04,2538.175300,123.614700,453.965300,5497.400900,773.708200,613.135600
2010-01-05,2587.340341,123.828444,456.643645,5442.659005,776.577233,613.333798
2010-01-06,2601.556308,121.858770,448.371256,5430.493033,791.397871,609.569715
2010-01-07,2653.088412,121.633521,440.742998,5433.535443,801.118742,603.230194
2010-01-08,2646.573033,122.442123,452.676971,5406.165282,797.931722,607.390475
...,...,...,...,...,...,...
2023-04-24,11796.630815,3141.635751,7201.740532,20908.287526,3679.543910,7313.069220
2023-04-25,11540.127848,3111.992285,6954.924494,20745.413500,3537.470391,7148.261806
2023-04-26,11335.595702,3111.802104,7118.338685,20655.079336,3488.990246,7666.044372


In [245]:
# get the daily total value for the protfolio 
daily_price["Total"]=daily_price.sum(axis=1)

# get the daily return rate
day1_price = daily_price.iloc[0]
daily_price["daily_return_rate_optimal"] = (daily_price["Total"] - day1_price["Total"]) / daily_price["Total"] * 100

daily_price

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft,Total,daily_return_rate_optimal
Date,,,,,,,,
2010-01-04,2538.175300,123.614700,453.965300,5497.400900,773.708200,613.135600,10000.000000,0.000000
2010-01-05,2587.340341,123.828444,456.643645,5442.659005,776.577233,613.333798,10000.382467,0.003825
2010-01-06,2601.556308,121.858770,448.371256,5430.493033,791.397871,609.569715,10003.246953,0.032459
2010-01-07,2653.088412,121.633521,440.742998,5433.535443,801.118742,603.230194,10053.349311,0.530662
2010-01-08,2646.573033,122.442123,452.676971,5406.165282,797.931722,607.390475,10033.179607,0.330699
...,...,...,...,...,...,...,...,...
2023-04-24,11796.630815,3141.635751,7201.740532,20908.287526,3679.543910,7313.069220,54040.907754,81.495500
2023-04-25,11540.127848,3111.992285,6954.924494,20745.413500,3537.470391,7148.261806,53038.190324,81.145661
2023-04-26,11335.595702,3111.802104,7118.338685,20655.079336,3488.990246,7666.044372,53375.850446,81.264936


In [246]:
daily_price.to_csv('dailyreturn.csv', index=True)

# Return Rate if only invest in one of the stock


In [247]:
stock_pd.head()

,JP Morgan,Apple,Amazon,Walmart,Target,Microsoft
Date,,,,,,
2010-01-04,30.279612,6.496294,6.6950,40.013077,34.240273,23.572369
2010-01-05,30.866135,6.507527,6.7345,39.614635,34.367241,23.579988
2010-01-06,31.035727,6.404015,6.6125,39.526085,35.023125,23.435276
2010-01-07,31.650488,6.392178,6.5000,39.548229,35.453320,23.191549
2010-01-08,31.572762,6.434672,6.6760,39.349014,35.312279,23.351494


In [248]:
one_stock = 10000/stock_pd